In [2]:
#Importing libraries
import numpy as np
import sys
import argparse
import numpy as np
import scipy.stats as ss
import math

In [20]:
# Enter the data file name, open it and assign it to the variable data_file
print ("Naive Bayes")
while True:
    try:
        filename = input("Enter the name of the data file:")       
        
    # In case of a error the except block will be executed
    except:
        print("Error: Could not find file or read data")
        continue
        
    # This block will be executed after correct data file name is entered
    else:
        print("Correct data file found and imported\n")
        print("----------------------------------------------------------------------------------------\n")
        break

Naive Bayes
Enter the name of the data file:project3_dataset1.txt
Correct data file found and imported

----------------------------------------------------------------------------------------



In [21]:
with open(filename, "r") as file:
        matrix = [line.rstrip().split('\t') for line in file]

In [22]:
matrix = np.array(matrix)
output_class = matrix[:,-1].reshape((matrix.shape[0],1)).astype(int)
matrix = matrix[:,0:-1]
#print(matrix) 

In [23]:
#To check if attribute is String or not
def is_string(s):
    try:
        complex(s) # for int, long, float and complex
    except ValueError:
        return True
    return False

In [24]:
s_indices = []
i=0
while i < len(matrix[0]):
    if is_string(matrix[0][i]):
        s_indices.append(i)
    i+=1       
for i in s_indices:
    unique_strings = np.unique(matrix[:,i])
    replacement_vals = list(range(len(unique_strings)))
    dictionary = dict(zip(unique_strings, replacement_vals))
    j=0
    while j < len(matrix[:,i]):
        matrix[j][i] = dictionary.get(matrix[j][i])
        j+=1

In [25]:
#Appending the output class at the last 
matrix = matrix.astype(float)
matrix = np.append(matrix,output_class,axis=1)

In [26]:
def prior_probability_zero(matrix):
    column = list(matrix[:,-1]).count(0)
    return float(column)/len(matrix)

In [27]:
def prior_probability_one(matrix):
    column = list(matrix[:,-1]).count(1)
    return float(column)/len(matrix)

In [28]:
#Mean of zeros
def meanzero(train_float):
        mean_zero=[]
        for row in train_float:
               if row[-1] == 0:
                    mean_zero.append(row[0:-1]) 
                
        return np.mean(mean_zero,axis = 0)

In [29]:
#Mean of ones
def meanone(train_float):
        mean_one=[]
        for row in train_float:
               if row[-1] == 1:
                    mean_one.append(row[0:-1]) 
                
        return np.mean(mean_one,axis = 0)

In [30]:
#Standard Deviation of zeros
def stdzero(train_float):
        std_zero = []
        for row in train_float:
               if row[-1] == 0:
                      std_zero.append(row[0:-1]) 
        return np.std(std_zero,axis = 0)

In [31]:
cross_validation = np.array_split(matrix, 10)

In [32]:
#Standard Deviation of ones
def stdone(train_float):
    return np.std([row[0:-1] for row in train_float if row[-1] == 1], axis = 0)

In [33]:
def  calculate_categorical(class_zero,class_one,test,train):
    string_prob_zero = np.empty(test.shape[0])
    string_prob_one = np.empty(test.shape[0])
    if len(s_indices) == 0:
        string_test = 0
    elif len(s_indices) != 0:
        string_prior_prob_one = {}
        string_prior_prob_zero = {}    
        
        for j in s_indices:
            string_prior_prob_one[j] = {}
            string_prior_prob_zero[j] = {}
            for k in np.unique(train[:,j]): 
                prior_one = float(list(class_one[:,j]).count(k))/list(class_one[:,-1].astype(int)).count(1) 
                prior_zero = float(list(class_zero[:,j]).count(k))/list(class_zero[:,-1].astype(int)).count(0)
                string_prior_prob_zero[j][k] = prior_zero
                string_prior_prob_one[j][k] = prior_one
       
    string_prob_one.fill(1.0)
    string_prob_zero.fill(1.0)
 
    if len(s_indices) != 0:
        t = 0
        while t < len(test):
            for i in s_indices:
                string_prob_one[t] *= string_prior_prob_one[i][test[t][i]]
                string_prob_zero[t] *= string_prior_prob_zero[i][test[t][i]] 
                t+=1
    return string_prob_one,string_prob_zero             

In [34]:
def calculate_probability(test, train):
    test_class = []
    
    class_zero = []
    class_one = []
    for j in range(len(train)):
        if int(train[j,-1]) == 0:
            class_zero.append(train[j,:])
        elif int(train[j,-1]) == 1:
            class_one.append(train[j,:])

    class_zero = np.asarray(class_zero)
    class_one = np.asarray(class_one)
    
    #Categorical Data
    string_prob_one,string_prob_zero = calculate_categorical(class_zero,class_one,test,train)

    #Calculating prior probabilities of zeros and ones
    prior_probablity_zero = prior_probability_zero(train) 
    prior_probablity_one = prior_probability_one(train) 

    #Excluding categorical
    final_itemset =[]
    for i in range(len(train[0])):
         if i not in s_indices:
                final_itemset.append(train[:,i])
    train_float = np.transpose(np.asarray(final_itemset))  
    
    mean_zero = meanzero(train_float)  
    mean_one = meanone(train_float) 

    std_zero = stdzero(train_float)      
    std_one =  stdone(train_float)      

    testing_float =[]
    for i in range(len(test[0])):
         if i not in s_indices:
                testing_float.append(test[:,i])
    test_float = np.transpose(np.asarray(testing_float))  

    #Actual probability of zero and one
    prob_zero = prior_probablity_zero * np.multiply(np.prod(np.exp(-1 * np.multiply(test_float[:,0:-1] - mean_zero,test_float[:,0:-1] - mean_zero) / (2 * np.multiply(std_zero,std_zero)))/(math.sqrt(math.pi*2)*std_zero), axis = 1), string_prob_zero)
    prob_one = prior_probablity_one * np.multiply(np.prod(np.exp(-1 * np.multiply(test_float[:,0:-1] - mean_one,test_float[:,0:-1] - mean_one) / (2 * np.multiply(std_one,std_one)))/(math.sqrt(math.pi*2)*std_one), axis = 1), string_prob_one)

    #Decide which test data point belongs to which class
    test_class = []
    for i in range(len(test)):
                if prob_one[i] > prob_zero[i]:
                       test_class.append(1)
                elif prob_one[i] < prob_zero[i]:
                       test_class.append(0)
    return test_class

In [35]:
#Calculating the performance parameters and Getting metrics - TruePositive, FalsePositive, TrueNegative, FalseNegative for calculating accuracy/precision/recall/f-measure
def performance_metrics(actual, predicted):
    tp = tn = fp = fn = 0
    for i in range(len(actual)):

        if actual[i] == 1 and predicted[i] == 1:
            tp += 1
        if actual[i] == 1 and predicted[i] == 0:
            fn += 1
        if actual[i] == 0 and predicted[i] == 1:
            fp += 1
        if actual[i] == 0 and predicted[i] == 0:
            tn += 1
    precision = 0
    recall = 0
    F1 = 0
    accuracy = 0

    if tp + fp != 0:
        precision = tp / float(tp + fp)
    if tp + fn != 0:
        recall = tp / float(fn + tp)
    if tp + tn + fp + fn != 0:
        accuracy = (tp + tn) / float(tp + tn + fp + fn)
    if precision + recall != 0:
        F1 = (2 * precision * recall) / float(precision + recall)

    precision = precision * 100
    recall = recall * 100
    accuracy = accuracy * 100
    F1 = F1 * 100
    print("----------")
    print(accuracy)
    print(precision)
    print(recall)
    print(F1)
    print("---------")
    return {'precision': precision, 'recall': recall, 'accuracy': accuracy, 'F1': F1}

In [36]:
scores = []
for i in range(len(cross_validation)):
    test_set = cross_validation[i]
    train_set=[]
    for l,x in enumerate(cross_validation):
         if l != i:
            train_set.append(x)
    train_set = np.vstack(train_set)
    test_set = np.asarray(test_set)
    train_set = np.asarray(train_set)
    class_list = calculate_probability(test_set, train_set)
    
    actual_class = test_set[:,-1]
    predicted_class = np.asarray(class_list)
    accuracy = performance_metrics(actual_class, predicted_class)
    scores.append(accuracy)

precision = 0
recall = 0
accuracy = 0
F1 = 0

for i in range(len(scores)):
    precision += scores[i]['precision']
    recall += scores[i]['recall']
    accuracy += scores[i]['accuracy']
    F1 += scores[i]['F1']

print('Final Accuracy: %.3f%%' % (accuracy/float(len(scores))))
print('Final Precision: %.3f%%' % (precision/float(len(scores))))
print('Final Recall: %.3f%%' % (recall/float(len(scores))))
print('Final F1: %.3f%%' % (F1/float(len(scores))))

----------
94.73684210526315
91.66666666666666
95.65217391304348
93.61702127659574
---------
----------
92.98245614035088
88.23529411764706
88.23529411764706
88.23529411764706
---------
----------
96.49122807017544
87.5
100.0
93.33333333333333
---------
----------
91.22807017543859
90.0
85.71428571428571
87.80487804878048
---------
----------
89.47368421052632
88.88888888888889
80.0
84.21052631578948
---------
----------
96.49122807017544
100.0
92.5925925925926
96.15384615384615
---------
----------
96.49122807017544
95.45454545454545
95.45454545454545
95.45454545454545
---------
----------
96.49122807017544
93.33333333333333
93.33333333333333
93.33333333333333
---------
----------
91.22807017543859
96.42857142857143
87.09677419354838
91.52542372881356
---------
----------
89.28571428571429
86.36363636363636
86.36363636363636
86.36363636363636
---------
Final Accuracy: 93.490%
Final Precision: 91.787%
Final Recall: 90.444%
Final F1: 91.003%
